In [ ]:
import json
import torch

# Load T5 model
# Load model directly
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
# Load one dialogue
with open("../data/mw24/dev_dials.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [76]:
# Pick a single turn from the first dialogue
turns = data[2]["dialogue"]
turn_idx = 3  # pick third turn (index 2)
history = turns[:turn_idx + 1]

In [77]:
# Format history
formatted_history = []

prompt = "You are a helpful assistant.\n\n"
prompt += "Here is the conversation history:\n\n"
for t in history:
    sys = t.get("system_transcript", "").strip()
    usr = t.get("transcript", "").strip()
    if sys:
        formatted_history.append(f"System: {sys}")
    if usr:
        formatted_history.append(f"User: {usr}")
prompt += "\n".join(formatted_history)
prompt += """\n\nSummarize key information from the conversation as a JSON dictionary.\n\n {"""
# prompt += """Example:\n{'train-leaveat': ..., 'train-destination': ..., 'train-day': ..., ...}
# """
prompt

'You are a helpful assistant.\n\nHere is the conversation history:\n\nUser: what can you tell me about the riverside brasserie ?\nSystem: it is a restaurant that serves modern european food near the centre of town . it is moderate -ly priced . the phone number is 01223259988 .\nUser: perfect . can you help me with a reservation for 6 people at 14:30 this coming sunday ? and please make sure i have a confirmation number to use .\nSystem: your reservation is set ! the table will be reserved for 15 minutes . your reference number is lzludtvi . is there anything else you need ?\nUser: i am also looking for a place to stay . in the south preferably .\nSystem: what price range were you thinking ?\nUser: no particular price range , but i would like it to be a 4 star hotel .\n\nSummarize key information from the conversation as a JSON dictionary.\n\n {'

In [78]:
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("=== Prompt ===")
# print(prompt)
# print("\n=== Thinking content ===")
# print(thinking_content)
print("\n=== Content ===")
print(content)

# Gold belief state
gold = {}
for slot_info in turns[turn_idx].get("belief_state", []):
    for slot, value in slot_info["slots"]:
        gold[slot] = value
print("\n=== Gold ===")
print(gold)




=== Content ===
```json
{
  "restaurant": {
    "name": "riverside brasserie",
    "location": "near the centre of town",
    "type": "modern European food",
    "price": "moderately priced",
    "phone": "01223259988"
  },
  "reservation": {
    "number_of_people": 6,
    "time": "14:30",
    "date": "sunday",
    "table_time": "15 minutes",
    "reference_number": "lzludtvi"
  },
  "hotel": {
    "location": "south",
    "price_range": "4-star",
    "description": "a 4-star hotel in the south"
  }
}
```

=== Gold ===
{'restaurant-book day': 'sunday', 'restaurant-book people': '6', 'restaurant-book time': '14:30', 'restaurant-food': 'modern european', 'restaurant-pricerange': 'moderate', 'restaurant-name': 'riverside brasserie', 'restaurant-area': 'centre', 'hotel-area': 'south', 'hotel-stars': '4'}
